In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from dbc.main import KmeansDiscreteMinmaxClassifier, CmeansDiscreteBayesianClassifier
from dbc.utils import compute_conditional_risk
from sklearn.model_selection import train_test_split

In [ ]:
num=1000
risks_DMC = np.zeros([num, 2])
risks_SPDBC = np.zeros([num, 2])

X, y = datasets.make_blobs(
    n_samples= [125 * 5 , 125 * 1],
    n_features=2,
    centers=[(9.5, 10), (10, 9.4)],
    cluster_std=[[0.6, 0.6], [0.35, 0.3]],
    shuffle=True,
    random_state=43
)

# 分割数据集，test_size=0.2表示测试集占20%
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,  # 测试集比例
    random_state=43,  # 随机种子，确保结果可重现
    stratify=y  # 保持分割后各类别的比例与原始数据集一致
)

SPDBC = CmeansDiscreteBayesianClassifier(n_clusters=15, fuzzifier=2)
SPDBC.fit(X_train, y_train)

# 创建prior_pred矩阵
prior_values = np.linspace(0, 1, num)  # 生成0到1之间的100个值
prior_pred = np.zeros((num, 2))
prior_pred[:, 0] = prior_values
prior_pred[:, 1] = 1 - prior_values  # 确保每行和为1

# 计算不同prior下的风险
risks = np.zeros((num, 2))
for i in range(num):
    y_pred = SPDBC.predict(X_train, prior_pred=prior_pred[i, :].reshape(1, -1))
    risk = compute_conditional_risk(y_train, y_pred)
    risks[i,:] = risk[0]
r = prior_pred[:, 0] * risks[:,0]  + prior_pred[:, 1] * risks[:,1]  # 贝叶斯风险

plt.figure(figsize=(10, 6))
plt.plot(prior_values, risks[:,0], 'r-', label='0 CCR')  # 0类类条件风险
plt.plot(prior_values, risks[:,1], 'b-', label='1 CCR')  # 1类类条件风险

Objective_Function = (risks[:,0] - r)**2 + (risks[:,1] - r)**2
A_Function = (risks[:,0] - risks[:,1])**2
B_Function = 1 - 2 * prior_pred[:,0] + 2 * prior_pred[:,0]**2

plt.plot(prior_values, Objective_Function, label='Objective Function=A*B')
plt.plot(prior_values, A_Function, label='Function A')
plt.plot(prior_values, B_Function, label='Function B')

plt.legend()

In [ ]:
Objective_Function[570]

In [ ]:
from typing import Tuple, Dict, Any
def risk_vector_from_pi(SPDBC, X_train, y_train, prior):
    y_pred = SPDBC.predict(X_train, prior_pred=prior.reshape(1, -1))
    R = compute_conditional_risk(y_train, y_pred)
    return R[0]

def quad_piece_from_R(R: np.ndarray) -> Tuple[float, float, np.ndarray]:
    """
    给定风险向量 R，返回凸二次片段的参数：
      f_R(pi) = K * (pi^T R - mu)^2 + c
    输出: (K, mu, c)，其中 K 是类别数（标量），mu=(1/K)*sum R_k（标量），c 常数项。
    """
    K = R.shape[0]
    mu = R.mean()
    c = (R @ R) - K * (mu ** 2)
    return K, mu, c

import cvxpy as cp
def solve_convex_upper_envelope(X_train, y_train, K, pi_init, max_iters=300, l2_reg: float = 0,lower_bound: float = 1e-9, verbose: bool = True, tol: float = 1e-13):
    # 1) 活跃约束集合：每条约束由一个 R 向量定义
    active_Rs = []
    # 2) 初始化：用 pi_init 生成第一条约束
    pi_curr = pi_init.copy()
    R0 = risk_vector_from_pi(SPDBC, X_train, y_train, pi_curr)
    active_Rs.append(R0)
    # 3) cvxpy 变量与问题骨架
    pi_var = cp.Variable(K)
    t_var = cp.Variable(1)
    constraints = []
    # 单纯形 + 下界
    constraints += [cp.sum(pi_var) == 1,
                    pi_var >= lower_bound]
    # 建立一个函数：根据 active_Rs 刷新 “t >= K*(pi^T R - mu)^2 + c” 约束
    def rebuild_piece_constraints():
        piece_constr = []
        for R in active_Rs:
            Kc, mu, c = quad_piece_from_R(R)
            piece_constr.append(t_var[0] >= R - pi_var @ R)
            piece_constr.append(-t_var[0] <= R - pi_var @ R)
        return piece_constr
    # 目标：min t (+ 可选 L2 正则)
    objective = cp.Minimize(t_var[0])
    # 初始拼装
    piece_constr = rebuild_piece_constraints()
    problem = cp.Problem(objective, constraints + piece_constr)
    history = []
    for it in range(max_iters):
        # 解当前凸问题
        problem.solve(verbose=False)  # OSQP/ECOS/SCS 都可；若报错换一个
        if problem.status not in ("optimal", "optimal_inaccurate"):
            raise RuntimeError(f"Solver failed: {problem.status}")

        pi_curr = pi_var.value
        t_curr = float(t_var.value[0])

        # 用新的 pi 计算“真实最坏片段”（即基于该 pi 产生的新 R）
        R_new = risk_vector_from_pi(SPDBC, X_train, y_train, pi_curr)
        Kc, mu, c = quad_piece_from_R(R_new)
        f_new = Kc * (pi_curr @ R_new - mu) ** 2 + c  # 该片段下的真实值

        violation = f_new - t_curr  # 若 > 0，说明这个“真正活跃”的片段约束未加入
        history.append((t_curr, float(violation), len(active_Rs)))

        if verbose:
            print(f"[iter {it:02d}] t={t_curr:.6e}, violation={violation:.3e}, |A|={len(active_Rs)}")

        if violation < tol:
            # 所有片段在 pi_curr 处都不比 t_curr 大 => 全局最优
            break

        # 否则把新片段加入，重建约束并继续
        # 去重：若 R_new 与已有约束非常接近就不重复加入
        if not any(np.allclose(R_new, R_old, atol=1e-12) for R_old in active_Rs):
            active_Rs.append(R_new)
            # 重新搭建“片段约束”
            piece_constr = rebuild_piece_constraints()
            problem = cp.Problem(objective, constraints + piece_constr)

    return pi_curr, t_curr, history

In [ ]:
index=500
R = risk_vector_from_pi(SPDBC, X_train, y_train, prior_pred[index, :])
R

In [ ]:
prior_pred[index, 0] * risks[index,0]  + prior_pred[index, 1] * risks[index,1]

In [ ]:
prior_pred[index, :].T @ R

In [ ]:
Kc, mu, c = quad_piece_from_R(R)
Kc * (prior_pred[index, :].T @ R - mu)**2 + c

In [ ]:
prior_pred[index, :]

In [ ]:
R - prior_pred[index, :] @ R

In [ ]:
np.sum((R - prior_pred[index, :] @ R)**2)

In [ ]:
K=2
pi_init = np.ones(K) / K
pi_init

In [ ]:
solve_convex_upper_envelope(X_train,y_train,K,prior_pred[1,:])

In [ ]:
prior_pred[1,:]